# EVALUATION LOOP

In [1]:
# ------------------------------------------------------------
# 0.  Imports & config – nothing here should clash with yours
# ------------------------------------------------------------
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
K       = 3                # demos per query
NUM_SAMPLES = 5             # model completions per query
TEMPERATURE = 0.7           # keep same as training loop

# ------------------------------------------------------------
# 1.  Initialise encoder **in eval mode** (weights frozen)
# ------------------------------------------------------------
encoder = MathBERTEncoder(device=DEVICE, trainable=False)
encoder.eval()                                   # no grads!

icl_model = OpenAIICLModel(api_key=API_KEY,
                           model_name="gpt-4.1-nano",
                           temperature=TEMPERATURE)

# ------------------------------------------------------------
# 2.  Load/define the inference set
#     If you already saved a slice elsewhere, just load it.
# ------------------------------------------------------------
gsm8k_data = load_dataset("gsm8k", "main")["train"].select(range(256))  # slice first 200 examples
gsm8k_to_infer = load_dataset("gsm8k", "main")["train"] \
                     .select(range(5))          # ⬅ change as needed

# ------------------------------------------------------------
# 3.  Evaluation loop
# ------------------------------------------------------------
total, correct = 0, 0
all_losses     = []          # optional – to compare prompt quality

for idx in tqdm(range(len(gsm8k_to_infer)), desc="Baseline eval"):
    item       = gsm8k_to_infer[idx]
    Q_inf, A_gt = item["question"], item["answer"]

    # Build demo pool (everything except current query)
    demos = [(d["question"], d["answer"])
             for j, d in enumerate(gsm8k_data) if j != idx]

    # Encode query + candidate demos
    with torch.no_grad():
        q_emb     = encoder.encode([Q_inf], detach=True).squeeze(0)
        demo_embs = encoder.encode([q for (q, _) in demos], detach=True)

    # ------- ORIGINAL cosine retrieval -------------------------
    top_k, _ = rc.retrieve_top_k_cosine(
        q_emb, demo_embs, k=min(K, len(demos))
    )
    print(f"🔍 Top-K Indices: {top_k}")
    selected_demos = [demos[i] for i in top_k]          # length = 2

    # ------- Run the ICL model --------------------------------
    responses_nested = sample_responses_per_demo(
        demo_tuples = selected_demos,
        Q_inf       = Q_inf,
        icl_model   = icl_model,
        num_samples = NUM_SAMPLES,
        parallel=True
    )
    print(responses_nested)
    flat_responses = [r for demo_resps in responses_nested for r in demo_resps]
    acc = compute_demo_accuracy(flat_responses, A_gt)
    print(acc) 
    correct += acc
    total   += 1

baseline_acc = correct / total
print(f"\n📊  Baseline accuracy on GSM8K_to_infer: {baseline_acc:.3%}")


c:\Users\abhin\miniconda3\envs\lima\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Baseline eval:   0%|          | 0/5 [00:00<?, ?it/s]

🔍 Top-K Indices: [81, 159, 35]


Baseline eval:   0%|          | 0/5 [00:02<?, ?it/s]

OUTPUT:  ('72', None)
OUTPUT:  ('48 + 48/2 = 48 + 24 = 72', None)
OUTPUT:  ('144', None)
OUTPUT:  ('72', None)
OUTPUT:  ('48 + (48/2) = 48 + 24 = 72', None)


AttributeError: 'str' object has no attribute 'answer'

# TRAINING LOOP

In [ ]:
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

# === Settings ===
API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
K = 16
NUM_SAMPLES_PER_DEMO = 6
LEARNING_RATE =  1e-5
MAX_STEPS = 10
TEMPERATURE = 0.7

# === Init ===
encoder = MathBERTEncoder(device=DEVICE, trainable=True)
encoder.train()

icl_model = OpenAIICLModel(api_key=API_KEY, model_name="gpt-4.1-nano", temperature=TEMPERATURE)
optimizer = torch.optim.Adam(encoder.parameters(), lr=LEARNING_RATE)

gsm8k_data = load_dataset('gsm8k', 'main')['train']
gsm8k_data = gsm8k_data.select(range(256))  # slice first 200 examples
gsm8k_data_to_infer = gsm8k_data.select(range(0,20))  # slice first 200 examples

# === Training Loop ===
for step in tqdm(range(MAX_STEPS), desc="Training Steps"):
    print(f"\n=== Training Step {step+1} ===")

    for inference_index in tqdm(range(len(gsm8k_data_to_infer)), desc="Examples"):
        inference_item = gsm8k_data_to_infer[inference_index]
        demo_pool = [d for idx, d in enumerate(gsm8k_data) if idx != inference_index]

        Q_inf = inference_item["question"]
        A_gt = inference_item["answer"]
        demos = [(d["question"], d["answer"]) for d in demo_pool]

        q_emb = encoder.encode([Q_inf], detach=False).squeeze(0)
        
        # SWITCHING TO BATCH ENCODE
        # demo_embs = encoder.encode([q for (q, a) in demos], detach=False)
        demo_questions = [q for (q, a) in demos]
        demo_embs = encoder.batched_encode(demo_questions, batch_size=16, detach=False)

        
        top_k_indices, _ = rc.retrieve_top_k_cosine(q_emb, demo_embs, k=min(K, len(demos)))
        print("-------Question---------")
        print(Q_inf)
        print(f"\n🧠 Inference Index {inference_index}")
        print(f"🔍 Top-K Indices: {top_k_indices}")
        
        top_k_indices, similarities = rc.retrieve_sample_k_cosine(q_emb, demo_embs, k=min(K, len(demos)))
        selected_demos = [demos[i] for i in top_k_indices]



        all_responses = sample_responses_per_demo(
            demo_tuples=selected_demos,
            Q_inf=Q_inf,
            icl_model=icl_model,
            num_samples=NUM_SAMPLES_PER_DEMO,
            parallel=True
        )

        rewards = []
        for i, responses in enumerate(all_responses):
            reward = compute_demo_accuracy(responses, A_gt)
            rewards.append(reward)
            # print(f"    Demo {i} | Reward: {reward:.2f}")

        rewards = torch.tensor(rewards, dtype=torch.float32).to(DEVICE)

        loss = grpo_step(
            rewards,
            similarities,
            q_emb,
            demo_embs,
            optimizer
        )

        print(f"✅ Loss: {loss:.4f}")



c:\Users\abhin\miniconda3\envs\lima\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Training Steps:   0%|          | 0/10 [00:00<?, ?it/s]


=== Training Step 1 ===


-------Question---------
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

🧠 Inference Index 0
🔍 Top-K Indices: [81, 159, 30, 84, 104, 145, 109, 204, 50, 4, 122, 8, 220, 56, 136, 51, 3, 99, 211, 176, 40, 251, 147, 82, 203, 60, 194, 150, 89, 239, 59, 207]
OUTPUT:  ('72', None)
OUTPUT:  ('72', None)
OUTPUT:  ('72', None)
OUTPUT:  ('72', None)
OUTPUT:  ('72', None)
OUTPUT:  ('48 + (48/2) = 48 + 24 = 72', None)
------------Demo 1--------
Q: Irene earns $500 if she works for 40 hours a week and gets an extra $20 for every hour of overtime. If she worked 50 hours last week, calculate her total income.
A: If Irene worked 50 hours last week, the total number of hours counting as overtime is 50-40 = <<50-40=10>>10 hours.
Since she's given $20 for every hour of overtime, she earned 10*$20 = $<<10*20=200>>200 in overtime.
Her total income, including the overtime, is $500+$200= $<<500+200=700

✅ Loss: -0.2100
-------Question---------
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

🧠 Inference Index 1
🔍 Top-K Indices: [177, 14, 12, 76, 155, 207, 221, 32, 60, 75, 210, 145, 64, 92, 142, 43, 201, 189, 164, 197, 62, 8, 174, 233, 168, 47, 204, 184, 95, 40, 205, 37]
OUTPUT:  ('10', None)
OUTPUT:  ('10', None)
OUTPUT:  ('10', None)
OUTPUT:  ('10', None)
OUTPUT:  ('10', None)
OUTPUT:  ('$10', None)
------------Demo 1--------
Q: Ian won $100 in the lottery.  He decided to use the money to pay off debts. He paid $20 to Colin. He then paid twice as much to Helen, as he had paid to Colin. Then finally, he paid half as much to Benedict, as he had paid to Helen.  How much money, in dollars, does he have left after paying off debts?
A: Starting with $100, he paid $20 to Colin, leaving him with $100-$20=$<<100-20=80>>80.
Twice as much as $20 is 2*$20=$<<2*20=40>>40.
Thus, he paid $40 to Helen, leaving him with $80-$40=$<<80-4

✅ Loss: -0.0000
-------Question---------
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?

🧠 Inference Index 2
🔍 Top-K Indices: [72, 229, 147, 177, 201, 32, 226, 37, 207, 53, 231, 51, 9, 211, 19, 104, 153, 75, 238, 15, 127, 80, 14, 36, 70, 200, 8, 236, 129, 184, 138, 101]
OUTPUT:  ('50', None)
OUTPUT:  ('50', None)
OUTPUT:  ('50', None)
OUTPUT:  ('50', None)
OUTPUT:  ('50', None)
OUTPUT:  ('50', None)
------------Demo 1--------
Q: Bobby has 16 toy cars, and the number of cars he has increases by 50% every year. How many toy cars will Bobby have in three years?
A: In the first year, Bobby will acquire 16 * .5 = <<16*.5=8>>8 new cars.
After the first year, he will have a total of 16 + 8 = <<16+8=24>>24 cars.
In the second year, Bobby will acquire 24 * .5 = <<24*.5=12>>12 new cars

...

✅ Loss: -0.1099
-------Question---------
Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?

🧠 Inference Index 14
🔍 Top-K Indices: [3, 93, 119, 220, 5, 133, 208, 159, 203, 139, 146, 175, 104, 243, 113, 254, 128, 194, 56, 151, 120, 14, 148, 249, 22, 158, 78, 4, 202, 228, 116, 124]
OUTPUT:  ('3', None)
OUTPUT:  ('3', None)
OUTPUT:  ('20 minutes / 8 pages = 2.5 minutes per page  \n120 pages × 2.5 minutes per page = 300 minutes  \n300 minutes ÷ 60 = 5 hours', None)
OUTPUT:  ('20 minutes / 8 pages = 2.5 minutes per page  \n120 pages × 2.5 minutes per page = 300 minutes  \n300 minutes ÷ 60 = 5 hours', None)
OUTPUT:  ('20 minutes / 8 pages = 2.5 minutes per page  \n120 pages * 2.5 minutes = 300 minutes  \n300 minutes / 60 = 5 hours', None)
OUTPUT:  ('20 minutes / 8 pages = 2.5 minutes per page  \n120 pages × 2.5 minutes = 300 minutes  \n300 minutes ÷ 60 = 5 hours', None)
------------Demo 1--------
Q: An aquarium holds an equal number of clownfish 

✅ Loss: -0.0249
-------Question---------
James creates a media empire.  He creates a movie for $2000.  Each DVD cost $6 to make.  He sells it for 2.5 times that much.  He sells 500 movies a day for 5 days a week.  How much profit does he make in 20 weeks?

🧠 Inference Index 15
🔍 Top-K Indices: [177, 31, 238, 207, 71, 193, 30, 253, 92, 176, 60, 133, 210, 185, 149, 40, 36, 234, 80, 9, 122, 43, 66, 155, 237, 101, 215, 240, 222, 110, 190, 233]
OUTPUT:  ('7500', None)
OUTPUT:  ('Profit per DVD = Selling price - Cost price = (2.5 × 6) - 6 = 15 - 6 = 9 dollars  \nNumber of DVDs sold in 20 weeks = 500 DVDs/day × 5 days/week × 20 weeks = 500 × 5 × 20 = 50,000 DVDs  \nTotal profit = Profit per DVD × Total DVDs = 9 × 50,000 = 450,000 dollars', None)
OUTPUT:  ('Profit per DVD = (Selling price) - (Cost to make) = (2.5 * 6) - 6 = 15 - 6 = 9 dollars  \nNumber of DVDs sold per day = 500  \nNumber of days in 20 weeks = 20 * 5 = 100 days  \nTotal DVDs sold = 500 * 100 = 50,000  \nTotal profit from DVDs 

✅ Loss: -0.1047


Training Steps:   0%|          | 0/10 [21:29<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# ------------------------------------------------------------
# 0.  Imports & config – nothing here should clash with yours
# ------------------------------------------------------------
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
K       = 3                # demos per query
NUM_SAMPLES = 5             # model completions per query
TEMPERATURE = 0.7           # keep same as training loop

# ------------------------------------------------------------
# 1.  Initialise encoder **in eval mode** (weights frozen)
# ------------------------------------------------------------
encoder.eval()                                   # no grads!

icl_model = OpenAIICLModel(api_key=API_KEY,
                           model_name="gpt-4.1-nano",
                           temperature=TEMPERATURE)

# ------------------------------------------------------------
# 2.  Load/define the inference set
#     If you already saved a slice elsewhere, just load it.
# ------------------------------------------------------------
gsm8k_data = load_dataset("gsm8k", "main")["train"].select(range(256))  # slice first 200 examples
gsm8k_to_infer = load_dataset("gsm8k", "main")["train"] \
                     .select(range(5))          # ⬅ change as needed

# ------------------------------------------------------------
# 3.  Evaluation loop
# ------------------------------------------------------------
total, correct = 0, 0
all_losses     = []          # optional – to compare prompt quality

for idx in tqdm(range(len(gsm8k_to_infer)), desc="Baseline eval"):
    item       = gsm8k_to_infer[idx]
    Q_inf, A_gt = item["question"], item["answer"]

    # Build demo pool (everything except current query)
    demos = [(d["question"], d["answer"])
             for j, d in enumerate(gsm8k_data) if j != idx]

    # Encode query + candidate demos
    with torch.no_grad():
        q_emb     = encoder.encode([Q_inf], detach=True).squeeze(0)
        demo_embs = encoder.encode([q for (q, _) in demos], detach=True)

    # ------- ORIGINAL cosine retrieval -------------------------
    top_k, _ = rc.retrieve_top_k_cosine(
        q_emb, demo_embs, k=min(K, len(demos))
    )
    print(f"🔍 Top-K Indices: {top_k}")
    selected_demos = [demos[i] for i in top_k]          # length = 2

    # ------- Run the ICL model --------------------------------
    responses_nested = sample_responses_per_demo(
        demo_tuples = selected_demos,
        Q_inf       = Q_inf,
        icl_model   = icl_model,
        num_samples = NUM_SAMPLES,
        parallel=True
    )
    print(responses_nested)
    flat_responses = [r for demo_resps in responses_nested for r in demo_resps]
    acc = compute_demo_accuracy(flat_responses, A_gt)
    print(acc) 
    correct += acc
    total   += 1

baseline_acc = correct / total
print(f"\n📊  Baseline accuracy on GSM8K_to_infer: {baseline_acc:.3%}")


Baseline eval:   0%|          | 0/5 [00:00<?, ?it/s]

🔍 Top-K Indices: [49, 138, 8]
------------Demo 1--------
Q: Gerald spends $100 a month on baseball supplies. His season is 4 months long. He wants to use the months he's not playing baseball to save up by raking, shoveling, and mowing lawns. He charges $10 for each. How many chores does he need to average a month to save up for his supplies?
A: He needs to save up $400 because 4 x 100 = <<4*100=400>>400
He has 8 months to earn this money because 12 - 4 = <<12-4=8>>8
He needs to earn $50 a month because 400 / 8 = <<400/8=50>>50
He needs to do 5 tasks a month because 50 / 10 = <<50/10=5>>5
#### 5

Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
A:
72


Baseline eval:  20%|██        | 1/5 [00:01<00:06,  1.55s/it]

[['72', '72', '72', '72', '72'], ['48 + 24 = 72', '48 + (48 / 2) = 48 + 24 = 72', '48 + 24 = 72', '48 + 24 = 72', '48 + (48 / 2) = 48 + 24 = 72'], ['72', '72', '72', '72', '144']]
0.9333333333333333
🔍 Top-K Indices: [12, 177, 221]
------------Demo 1--------
Q: Jasper will serve charcuterie at his dinner party. He buys 2 pounds of cheddar cheese for $10, a pound of cream cheese that cost half the price of the cheddar cheese, and a pack of cold cuts that cost twice the price of the cheddar cheese. How much does he spend on the ingredients?
A: A pound of cream cheese cost $10 / 2 = $<<10/2=5>>5.
A pack of cold cuts cost $10 x 2 = $<<10*2=20>>20.
Jasper spent $10 + $5 + $20 = $<<10+5+20=35>>35 on the ingredients.
#### 35

Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
A:
10


Baseline eval:  40%|████      | 2/5 [00:03<00:04,  1.50s/it]

...

[['50', '50', '50', '50', '50'], ['55', '25', '25', '25', '70'], ['55', '100 - (50 + 15 + 30) = 100 - 95 = 5', '55', '55', '55']]
0.9333333333333333
🔍 Top-K Indices: [5, 49, 8]
------------Demo 1--------
Q: Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?
A: He eats 32 from the largest pizzas because 2 x 16 = <<2*16=32>>32
He eats 16 from the small pizza because 2 x 8 = <<2*8=16>>16
He eats 48 pieces because 32 + 16 = <<32+16=48>>48
#### 48

Q: Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?
A:
54


Baseline eval:  80%|████████  | 4/5 [00:07<00:02,  2.14s/it]

[['54', '36', '48', '33', '54'], ['48', '48', '48', '48', '48'], ['54', '36', '54', '54', '54']]
0.0
🔍 Top-K Indices: [206, 156, 175]
------------Demo 1--------
Q: John volunteers at a shelter twice a month for 3 hours at a time.  How many hours does he volunteer per year?
A: He volunteers 2*12=<<2*12=24>>24 times a year
So he volunteers for 24*3=<<24*3=72>>72 hours
#### 72

Q: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?
A:
624


Baseline eval: 100%|██████████| 5/5 [00:09<00:00,  1.92s/it]

[['624', '312', '624', '312', '624'], ['312', '312', '312', '312', '312'], ['3 pages × 2 friends × 2 times a week × 52 weeks = 3 × 2 × 2 × 52 = 624', '312', '312', '312', '312']]
0.26666666666666666

📊  Baseline accuracy on GSM8K_to_infer: 62.667%


In [ ]:
# Save the updated MathBERT model
save_path = "./updated_mathbert"  # your save directory
encoder.model.save_pretrained(save_path)
encoder.tokenizer.save_pretrained(save_path)

# LOADING

# from transformers import BertTokenizer, BertModel

# model = BertModel.from_pretrained("./updated_mathbert")
# tokenizer = BertTokenizer.from_pretrained("./updated_mathbert")
